In [17]:
"""
Code borrowed from: "Python Machine Learning" (2nd Edition) by Sebastian Raschka and Vahid Mirjalili
Dataset borrowed from: https://www.kaggle.com/oddrationale/mnist-in-csv
"""
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')

import os
print(os.listdir("C:/Books/Texas State Books/MNIST"))

['mnist_test.csv', 'mnist_test.csv.zip', 'mnist_train.csv', 'mnist_train.csv.zip']


In [18]:
# read train 
train = pd.read_csv("C:/Books/Texas State Books/MNIST/mnist_train.csv")
print(train.shape)
train.head()

(60000, 785)


,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# read test 
test= pd.read_csv("C:/Books/Texas State Books/MNIST/mnist_test.csv")
print(test.shape)
test.head()

(10000, 785)


,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# put labels into y_train variable
y_train = train["label"]
# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1)

# put labels into y_test variable
y_test = test["label"]
# Drop 'label' column
X_test = test.drop(labels = ["label"],axis = 1)

# Normalize the data
X_train_centered = X_train / 255.0
X_test_centered = X_test / 255.0
del X_train, X_test

print(X_train_centered.shape, y_train.shape)
print(X_test_centered.shape, y_test.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [21]:
# Set the random seeds for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

# One-Hot Encode the classes
y_train_onehot = keras.utils.to_categorical(y_train)

In [22]:
# Define the MLP model
model = keras.models.Sequential()

model.add(
    keras.layers.Dense( units=50,
        input_dim=X_train_centered.shape[1],
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'))

model.add(
    keras.layers.Dense( units=50,
        input_dim=50,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'))

model.add(
    keras.layers.Dense( units=y_train_onehot.shape[1],
        input_dim=50,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='softmax'))

In [23]:
sgd_optimizer = keras.optimizers.SGD(
                    lr=0.001, decay=1e-7, momentum=.9)

model.compile(optimizer=sgd_optimizer,
    loss='categorical_crossentropy')

# Train the ANN
history = model.fit(X_train_centered, y_train_onehot,
    batch_size=64,
    epochs=50,
    verbose=1,
    validation_split=0.1) # 90% training / 10% validation

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 3s 60us/sample - loss: 0.9660 - val_loss: 0.4692
Epoch 2/50
54000/54000 [==============================] - 2s 46us/sample - loss: 0.4600 - val_loss: 0.3363
Epoch 3/50
54000/54000 [==============================] - 2s 45us/sample - loss: 0.3732 - val_loss: 0.2876
Epoch 4/50
54000/54000 [==============================] - 2s 45us/sample - loss: 0.3314 - val_loss: 0.2601
Epoch 5/50
54000/54000 [==============================] - 2s 46us/sample - loss: 0.3045 - val_loss: 0.2409
Epoch 6/50
54000/54000 [==============================] - 2s 45us/sample - loss: 0.2845 - val_loss: 0.2287
Epoch 7/50
54000/54000 [==============================] - 3s 52us/sample - loss: 0.2681 - val_loss: 0.2136
Epoch 8/50
54000/54000 [==============================] - 2s 46us/sample - loss: 0.2539 - val_loss: 0.2039
Epoch 9/50
54000/54000 [==============================] - 3s 47us/sample - loss: 0.2415 - val_l

In [24]:
# Print the performance metrics
y_train_pred = model.predict_classes(X_train_centered, verbose=0)
correct_preds = np.sum(y_train == y_train_pred, axis=0)
train_acc = correct_preds / y_train.shape[0]

print('Training accuracy: %.2f%%' % (train_acc * 100))

y_test_pred = model.predict_classes(X_test_centered, verbose=0)
correct_preds = np.sum(y_test == y_test_pred, axis=0)
test_acc = correct_preds / y_test.shape[0]

print('Test accuracy: %.2f%%' % (test_acc * 100))

Training accuracy: 97.76%
Test accuracy: 96.80%
